In [1]:
import pandas as pd

file = pd.read_csv('./Input/data/no_mask/no_mask.txt')
file.head()

,user_id\toriginal_image\tface_id\tage\tgender\tx\ty\tdx\tdy\ttilt_ang\tfiducial_yaw_angle\tfiducial_score
30601258@N03\t10399646885_67c7d20df9_o.jpg\t1\t(25,32)\tf\t0\t414\t1086\t1383\t-115\t30\t17
30601258@N03\t10424815813_e94629b1ec_o.jpg\t2\t(25,32)\tm\t301\t105\t640\t641\t0\t0\t94
30601258@N03\t10437979845_5985be4b26_o.jpg\t1\t(25,32)\tf\t2395\t876\t771\t771\t175\t-30\t74
30601258@N03\t10437979845_5985be4b26_o.jpg\t3\t(25,32)\tm\t752\t1255\t484\t485\t180\t0\t47
30601258@N03\t11816644924_075c3d8d59_o.jpg\t2\t(25,32)\tm\t175\t80\t769\t768\t-75\t0\t34


In [9]:
import pandas as pd
import csv

file = pd.read_csv('./Input/data/no_mask/no_mask.txt', delimiter = '\t')
file.head()